<a href="https://colab.research.google.com/github/Ibraheem101/mlops/blob/main/foundations/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import re
import json
import math
import nltk
import torch
import random
import itertools
import collections
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
import matplotlib.pyplot as plt

In [2]:
seed = 1234

In [3]:
def set_seeds(seed=1234):
    """Set seeds for reproducibility."""
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # multi-GPU

In [4]:
set_seeds(seed = seed)

In [5]:
# Set device
cuda = True
device = torch.device("cuda" if (
    torch.cuda.is_available() and cuda) else "cpu")
torch.set_default_tensor_type("torch.FloatTensor")
if device.type == "cuda":
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
print (device)

cpu


In [6]:
# Load data
url = "https://raw.githubusercontent.com/GokuMohandas/Made-With-ML/main/datasets/news.csv"
df = pd.read_csv(url, header=0) # load
df = df.sample(frac=1).reset_index(drop=True) # shuffle
df.head()

,title,category
0,Sharon Accepts Plan to Reduce Gaza Army Operat...,World
1,Internet Key Battleground in Wildlife Crime Fight,Sci/Tech
2,July Durable Good Orders Rise 1.7 Percent,Business
3,Growing Signs of a Slowing on Wall Street,Business
4,The New Faces of Reality TV,World


### Preprocessing

In [7]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [8]:
nltk.download("stopwords")
stop_words = stopwords.words("english")
print (stop_words[:5])
sstm = SnowballStemmer('english')

['i', 'me', 'my', 'myself', 'we']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def preprocess(text, stopwords = stop_words):

    # lowercasing
    text = text.lower()

    # Remove stopwords
    pattern = re.compile(r"\b(" + r"|".join(stopwords) + r")\b\s*")
    text = pattern.sub("", text)

    # Remove words in parenthesis
    text = re.sub(r"\([^)]*\)", "", text)

    # Spacing and filters
    text = re.sub(r"([-;;.,!?<=>])", r" \1 ", text)  # separate punctuation tied to words
    text = re.sub("[^A-Za-z0-9]+", " ", text)  # remove non alphanumeric chars
    text = re.sub(" +", " ", text)  # remove multiple spaces
    text = text.strip()

    return text



In [10]:
# Sample
text = "US and China take a potentially crucial step"
preprocess(text=text)


'us china take potentially crucial step'

In [11]:
# Apply to dataframe
preprocessed_df = df.copy()
preprocessed_df.title = preprocessed_df.title.apply(preprocess)

preprocessed_df.head()

,title,category
0,sharon accepts plan reduce gaza army operation...,World
1,internet key battleground wildlife crime fight,Sci/Tech
2,july durable good orders rise 1 7 percent,Business
3,growing signs slowing wall street,Business
4,new faces reality tv,World


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
TRAIN_SIZE = 0.7
VAL_SIZE = 0.15
TEST_SIZE = 0.15

In [14]:
def train_val_test_split(X, y, train_size):
    """Split dataset into data splits."""
    X_train, X_, y_train, y_ = train_test_split(X, y, train_size=TRAIN_SIZE, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_, y_, train_size=0.5, stratify=y_)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [15]:
# Data
X = preprocessed_df["title"].values
y = preprocessed_df["category"].values

In [16]:
# Create data splits
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
    X=X, y=y, train_size=TRAIN_SIZE)
print (f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print (f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print (f"X_test: {X_test.shape}, y_test: {y_test.shape}")
print (f"Sample point: {X_train[10]} → {y_train[10]}")

X_train: (84000,), y_train: (84000,)
X_val: (18000,), y_val: (18000,)
X_test: (18000,), y_test: (18000,)
Sample point: air france klm sales rise 6 4 passenger increase → Business


### Label encoding

In [17]:
from collections import OrderedDict

In [18]:
class LabelEncoder(object):
    """
        Class to transform categorical labels into numerical values.

        Attributes:
            mapping (dict): A dictionary that maps labels to their corresponding numerical values.
            reverse_mapping (dict): A dictionary that maps numerical values back to their original labels.
            classes (list): A list of unique labels.

        Methods:
            fit(data): Fit the encoder to the given data by creating the mapping and reverse_mapping dictionaries.
            encode(data): Encode the given data by replacing labels with their corresponding numerical values.
            decode(data): Decode the given data by replacing numerical values with their original labels.
            __len__(): Return the number of unique labels in the encoder.
            __str__(): Return a string representation of the encoder.
            save(fp): Save the encoder's mapping dictionary to a JSON file.
            load(fp): Load a saved encoder from a JSON file.

    """


    def __init__(self):
        self.mapping = {}
        self.reverse_mapping = {}
        self.classes = []

    def fit(self, data):
        unique_labels = list(OrderedDict.fromkeys(data))
        for value, label in enumerate(unique_labels):
            self.mapping[label] = value
            self.reverse_mapping[value] = label
            self.classes.append(label)

    def encode(self, data):
        return [self.mapping[i] for i in data]

    def decode(self, data):
        return [self.reverse_mapping[j] for j in data]

    def __len__(self):
        return len(self.mapping)

    def __str__(self):
        return f"<LabelEncoder(num_classes={len(self)})>"

    def save(self, fp):
        with open(fp, "w") as fp:
            contents = {'class_to_index': self.class_to_index}
            json.dump(contents, fp, indent=4, sort_keys=False)

    @classmethod
    def load(cls, fp):
        with open(fp, "r") as fp:
            kwargs = json.load(fp=fp)
        return cls(**kwargs)

In [19]:
# Encode
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
num_classes = len(label_encoder)
label_encoder.mapping

{'World': 0, 'Sports': 1, 'Business': 2, 'Sci/Tech': 3}

In [20]:
# Convert labels to tokens
print (f"y_train[0]: {y_train[0]}")
y_train_enc = label_encoder.encode(y_train)
y_val_enc = label_encoder.encode(y_val)
y_test_enc = label_encoder.encode(y_test)
print (f"y_train[0]: {y_train_enc[0]}")

y_train[0]: World
y_train[0]: 0


### Class weights

In [21]:
# Calculate class weights
class_weights = {}
total_samples = len(y_train_enc)
num_classes = len(np.unique(y_train_enc))
class_samples = np.bincount(y_train_enc)
for i in range(num_classes):
    class_weights[i] = total_samples / (num_classes * class_samples[i])

print(f"Class weights: {class_weights}")

Class weights: {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0}


### Tokenizer

In [22]:
from collections import Counter
from more_itertools import take

In [23]:
class Tokenizer(object):
    def __init__(self, char_level, num_tokens=None,
                 pad_token="<PAD>", oov_token="<UNK>",
                 token_to_index=None):
        self.char_level = char_level
        self.separator = "" if self.char_level else " "
        if num_tokens: num_tokens -= 2 # pad + unk tokens
        self.num_tokens = num_tokens
        self.pad_token = pad_token
        self.oov_token = oov_token
        if not token_to_index:
            token_to_index = {pad_token: 0, oov_token: 1}
        self.token_to_index = token_to_index
        self.index_to_token = {v: k for k, v in self.token_to_index.items()}

    def __len__(self):
        return len(self.token_to_index)

    def __str__(self):
        return f"<Tokenizer(num_tokens={len(self)})>"

    def fit_on_texts(self, texts):
        if not self.char_level:
            texts = [text.split(" ") for text in texts]
        all_tokens = [token for text in texts for token in text]
        counts = Counter(all_tokens).most_common(self.num_tokens)
        self.min_token_freq = counts[-1][1]
        for token, count in counts:
            index = len(self)
            self.token_to_index[token] = index
            self.index_to_token[index] = token
        return self

    def texts_to_sequences(self, texts):
        sequences = []
        for text in texts:
            if not self.char_level:
                text = text.split(" ")
            sequence = []
            for token in text:
                sequence.append(self.token_to_index.get(
                    token, self.token_to_index[self.oov_token]))
            sequences.append(np.asarray(sequence))
        return sequences

    def sequences_to_texts(self, sequences):
        texts = []
        for sequence in sequences:
            text = []
            for index in sequence:
                text.append(self.index_to_token.get(index, self.oov_token))
            texts.append(self.separator.join([token for token in text]))
        return texts

    def save(self, fp):
        with open(fp, "w") as fp:
            contents = {
                "char_level": self.char_level,
                "oov_token": self.oov_token,
                "token_to_index": self.token_to_index
            }
            json.dump(contents, fp, indent=4, sort_keys=False)

    @classmethod
    def load(cls, fp):
        with open(fp, "r") as fp:
            kwargs = json.load(fp=fp)
        return cls(**kwargs)


In [24]:
# Tokenize
tokenizer = Tokenizer(char_level=False, num_tokens=500)
tokenizer.fit_on_texts(texts=X_train)
VOCAB_SIZE = len(tokenizer)
print (tokenizer)

<Tokenizer(num_tokens=500)>


In [25]:
# Sample of tokens
print (take(5, tokenizer.token_to_index.items()))
print (f"least freq token's freq: {tokenizer.min_token_freq}") # use this to adjust num_tokens

[('<PAD>', 0), ('<UNK>', 1), ('39', 2), ('b', 3), ('gt', 4)]
least freq token's freq: 166


In [26]:
# Convert texts to sequences of indices
X_train_tok = tokenizer.texts_to_sequences(X_train)
X_val_tok = tokenizer.texts_to_sequences(X_val)
X_test_tok = tokenizer.texts_to_sequences(X_test)
preprocessed_text = tokenizer.sequences_to_texts([X_train_tok[0]])[0]
print ("Text to indices:\n"
    f"  (preprocessed) → {preprocessed_text}\n"
    f"  (tokenized) → {X_train_tok[0]}")

Text to indices:
  (preprocessed) → china <UNK> north korea nuclear talks
  (tokenized) → [ 16   1 285 142 114  24]


### One-hot encoding

In [27]:
def one_hot_encode(features, num_classes):
    num_samples = len(features)

    # Create an array of zeros with shape (num_samples, num_classes)
    one_hot_encoded = np.zeros((num_samples, num_classes))

    # Set the corresponding element to 1 for each sample and class
    for i, value in enumerate(features):
        one_hot_encoded[i, value] = 1

    return one_hot_encoded

In [28]:
# One-hot encoding
print (X_train_tok[0])
print (len(X_train_tok[0]))
cat = one_hot_encode(features=X_train_tok[0], num_classes=len(tokenizer))
print (cat)
print (cat.shape)

[ 16   1 285 142 114  24]
6
[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(6, 500)


In [29]:
# Convert tokens to one-hot
vocab_size = len(tokenizer)
X_train_hot = [one_hot_encode(seq, num_classes=vocab_size) for seq in X_train_tok]
X_val_hot = [one_hot_encode(seq, num_classes=vocab_size) for seq in X_val_tok]
X_test_hot = [one_hot_encode(seq, num_classes=vocab_size) for seq in X_test_tok]

### Padding
In the context of Natural Language Processing (NLP), padding refers to the process of adding special tokens or characters to sequences in order to make them of equal length. It is necessary because many machine learning models require input data of consistent dimensions.

Padding is particularly relevant when working with sequential data, such as sentences or documents, where the length of the text varies. To ensure that all sequences have the same length, shorter sequences are padded with special tokens (such as <PAD>) to match the length of the longest sequence in the dataset.

In [34]:
len(X_train_hot[0]), len(X_train_hot[1]), len(X_train_hot[9]) # Varying lengths

(6, 5, 7)

In [35]:
def pad_sequences(sequences, max_seq_len=0):
    """Pad sequences to max length in sequence."""
    max_seq_len = max(max_seq_len, max(len(sequence) for sequence in sequences))
    num_classes = sequences[0].shape[-1]
    padded_sequences = np.zeros((len(sequences), max_seq_len, num_classes))
    for i, sequence in enumerate(sequences):
        padded_sequences[i][:len(sequence)] = sequence
    return padded_sequences

<details>
  <summary>Padding with zero</summary>

  Padding with 0 is a common practice and is generally a good idea in many cases. Here are a few reasons why padding with 0 is commonly used:

Compatibility: Padding with 0 is compatible with many machine learning frameworks and libraries. Most frameworks, including PyTorch and TensorFlow, handle 0-padding efficiently and have built-in functions to handle it.

Zero-padding doesn't introduce noise: When padding with zeros, you are essentially adding neutral values that do not carry any meaning or bias. This ensures that the padding does not introduce any unwanted noise or affect the data distribution.

Easy handling of variable-length sequences: Padding with 0 allows you to convert variable-length sequences into fixed-length sequences, which is often required for batch processing and model training. It simplifies the handling of sequences by ensuring consistent dimensions.

Memory efficiency: Padding with 0 doesn't require additional memory or storage for the padding values. Zeros are already present in the memory and can be easily allocated, making it memory-efficient.

However, it's important to note that the choice of padding value may depend on the specific problem and the data being used. In some cases, other padding values such as a specific value outside the range of the data or a special token may be more appropriate. It's always recommended to consider the characteristics of your data and the requirements of your model when deciding on the padding strategy.

</details>

In [38]:
# 3D sequences
print (X_train_hot[0].shape, X_train_hot[1].shape, X_train_hot[2].shape)
padded = pad_sequences(X_train_hot[0:3])
print (padded.shape)

(6, 500) (5, 500) (6, 500)
(3, 6, 500)
